## Mask R-CNN - Loss Fucntions


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import tensorflow as tf
import keras.backend as KB
import numpy as np
from mrcnn.datagen     import data_generator, load_image_gt
from mrcnn.callbacks   import get_layer_output_1,get_layer_output_2
from mrcnn.utils       import mask_string, parse_image_meta, apply_box_deltas_tf
from mrcnn.visualize   import display_gt_bboxes, display_roi_proposals, plot_3d_heatmap, plot_gaussian2
from mrcnn.visualize   import plot_bbox_heatmaps, plot_one_bbox_heatmap
from mrcnn.visualize   import display_gt_bboxes, display_roi_proposals
import mrcnn.visualize as visualize
from mrcnn.prep_notebook import prep_oldshapes_dev, prep_oldshapes_train

model_info = prep_oldshapes_train(init_with = 'last', FCN_layers = True, batch_sz = 16)
model, dataset_train, dataset_val, train_generator, val_generator, config =  model_info

In [ ]:
train_batch_x, train_batch_y = next(train_generator)

In [ ]:
# train_batch_x, train_batch_y = next(train_generator)
imgmeta_idx = model.keras_model.input_names.index('input_image_meta')
img_meta    = train_batch_x[imgmeta_idx]

for img_idx in range(config.BATCH_SIZE):
    image_id = img_meta[img_idx,0]
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    print('Image id: ',image_id)
    print('Image meta', img_meta[img_idx])
    print('Classes (1: circle, 2: square, 3: triangle ): ',class_ids)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

In [ ]:
model.layer_info()
# model.keras_model.outputs[0].name

In [ ]:
model_output = get_layer_output_2(model.keras_model, train_batch_x, 1)
# model_output = get_layer_output_1(model.keras_model, train_batch_x, [0,1,2,3,4,5,6,7], 1)


In [ ]:
print(len(model_output))

rpn_class_logits          = model_output[0]          # layer:  0   shape: (16, 4092, 2)
rpn_class                 = model_output[1]          # layer:  1   shape: (16, 4092, 2)
rpn_bbox                  = model_output[2]          # layer:  2   shape: (16, 4092, 4)
rpn_roi_proposals         = model_output[3]          # layer:  3   shape: (16, 2000, 4)
output_rois               = model_output[4]          # layer:  4   shape: (16, 32, 4)
target_class_ids          = model_output[5]          # layer:  5   shape: (16, 32)
target_bbox_deltas        = model_output[6]          # layer:  6   shape: (16, 32, 4)
roi_gt_boxes              = model_output[7]          # layer:  7   shape: (16, 32, 4)
mrcnn_class_logits                 = model_output[8]          # layer:  8   shape: (16, 32, 4)
mrcnn_class                 = model_output[9]          # layer:  9   shape: (16, 32, 4)
mrcnn_bbox                = model_output[10]          # layer: 10   shape: (16, 32, 4, 4)
rpn_class_loss            = model_output[11]          # layer: 11   shape: (1, 1)
rpn_bbox_loss             = model_output[12]          # layer: 12   shape: (1, 1)
mrcnn_class_loss          = model_output[13]          # layer: 13   shape: (1, 1)
mrcnn_bbox_loss           = model_output[14]          # layer: 14   shape: (1, 1)
pred_heatmap_norm         = model_output[15]          # layer: 15   shape: (16, 128, 128, 4)
gt_heatmap_norm           = model_output[16]          # layer: 16   shape: (16, 128, 128, 4)
pred_heatmap_scores       = model_output[17]          # layer: 17   shape: (16, 4, 32, 11)
gt_heatmap_scores         = model_output[18]          # layer: 18   shape: (16, 4, 32, 11)
pred_tensor               = model_output[19]          # layer: 19   shape: (16, 4, 32, 6)
gt_tensor                 = model_output[20]          # layer: 20   shape: (16, 4, 32, 6)
pred_heatmap              = model_output[21]          # layer: 21   shape: (16, 128, 128, 4)
gt_heatmap                = model_output[22]          # layer: 22   shape: (16, 128, 128, 4)
fcn_heatmap_norm          = model_output[23]          # layer: 23   shape: (16, 128, 128, 4)
fcn_heatmap_scores        = model_output[24]          # layer: 24   shape: (16, 4, 32, 16)
fcn_heatmap               = model_output[25]          # layer: 25   shape: (16, 128, 128, 4)
fcn_norm_loss             = model_output[26]          # layer: 26   shape: (1, 1)

for i in model_output:
    print( i.shape)

In [ ]:
input_image      =  train_batch_x[0]
input_image_meta =  train_batch_x[1]
input_rpn_match  =  train_batch_x[2]
input_rpn_bbox   =  train_batch_x[3]
input_gt_class_ids = train_batch_x[4]
input_gt_bboxes    = train_batch_x[5]
input_gt_masks     = train_batch_x[6]
print(' Input image shape is :', input_image.shape)
h, w = input_image.shape[1], input_image.shape[2]      #  tf.shape(input_image)[1], tf.shape(input_image)[2]
input_normlzd_gt_bboxes = tf.identity(input_gt_bboxes / [h,w,h,w])

# gt_masks   =  train_batch_x[6]
print(' input_rpn_match    ', input_rpn_match.shape)
print(' input_rpn_bbox     ', input_rpn_bbox.shape)
print(' input_gt_class_ids ', input_gt_class_ids.shape)
print(' input_gt_bboxes    ', input_gt_bboxes.shape)
print(' input_normlzd_gt_bboxes    ', input_normlzd_gt_bboxes.shape)


### Review values  

####  display Pred_Tensor, Pred_heatmap, mrcnn_class

In [ ]:
np.set_printoptions(precision=4, threshold=None, linewidth=120, suppress=True)
img = 1
print('input_gt_class_ids')
print(input_gt_class_ids.shape)
print(input_gt_class_ids)


print('input_gt_bboxes')
print(input_gt_bboxes.shape)
print(input_gt_bboxes[img,:5]/[128,128,128,128])

print(target_class_ids.shape)
print(target_class_ids)
# print(input_gt_class_ids[0])

print(' roi_gt_bboxes')
print(roi_gt_boxes.shape)
print(roi_gt_boxes[img, :20])

# print(' gt_tensor')
# print(gt_tensor.shape)
# print(gt_tensor[img,:,:10])

# print(' output_rois')
# print(output_rois.shape)
# print(output_rois[img,:15] * [128, 128,128,128])


# print(' roi_gt_boxes')
# print(roi_gt_boxes.shape)
# print(roi_gt_boxes[img,:15] * [128, 128,128,128])

# print(' Pred Heatmap Scores')
# print(pred_heatmap_scores.dtype)
# print(pred_heatmap_scores[img,:,:10])

# print(' FCN Scores')
# print(fcn_scores.dtype)
# print(fcn_scores[img,:,:10, 4:])

# img = 2
# max_score = np.max(mrcnn_class, axis = -1)
# max_class = np.argmax(mrcnn_class, axis = -1)
# # print(' output_rois[',img,'] \n', output_rois[1]*[128,128,128,128])
# print('max class shape:',max_class.shape, 'max score shape: ',max_score.shape)
# print('max class[',img,']\n',max_class[img])
# print('max score[',img,']\n',max_score[img])
# print('mrcnn class.shape ',mrcnn_class.shape)
# print('mrcnn_class[',img,',:]\n',mrcnn_class[img,:])
# print(output_rois[1])
 


####  Display `output_rois` for visual check - passed on to  `build_pred_tensor()`

In [ ]:
np.set_printoptions(linewidth=150, precision=6)
# print('scatter shape is ', pred_scatt.get_shape())
print('output_rois shape is ', output_rois.shape)
img = 0
for img in range(5):
    print('Image ', img , ' ------------')
    print(output_rois[img])


####  Display for visual check - `pred_tensor` is the final result which is passed on to  `build_gaussian_tf()`

In [ ]:
# with sess.as_default():
np.set_printoptions(linewidth=150, precision=6)
# print('scatter shape is ', pred_scatt.get_shape())
print('pred_tensor shape is ', pred_tensor.shape)
img = 0
for k in range(4):
    print('Image ', img , '/ Class ',k,' ------------')
    print(pred_heatmap_scores[img,k])


####  Display for visual check - `gt_tensor` is the final result which is passed on to  `build_gaussian_tf()`

In [ ]:
np.set_printoptions(linewidth=150, precision=6)
print('gt_tensor shape is ', gt_tensor.shape)
img = 1
for k in range(4):
    print('Image ', img , '/ Class ',k,' ------------')
    print(gt_tensor[img,k])


####  Display for visual check - `pred_heatmap_scores` is the final result which is passed on to  `build_gaussian_tf()`

In [ ]:
# with sess.as_default():
np.set_printoptions(linewidth=150, precision=6)

# print('scatter shape is ', pred_scatt.get_shape())
print('pred_heatmap_score shape is ', pred_heatmap_scores.shape)
img = 0
for k in range(4):
    print('Image ', img , '/ Class ',k,' ------------')
    print(pred_heatmap_scores[img,k])


####  Display for visual check - `gt_heatmap_scores` is the final result which is passed on to  `build_gaussian_tf()`

In [ ]:
# with sess.as_default():
np.set_printoptions(linewidth=150, precision=6)
# print('scatter shape is ', pred_scatt.get_shape())
print('gt_heatmap_score shape is ', gt_heatmap_scores.shape)
img = 0
for k in range(4):
    print('Image ', img , '/ Class ',k,' ------------')
    print(gt_heatmap_scores[img,k])


####  Display for visual check - `fcn_heatmap_scores` is the final result which is passed on to  `build_gaussian_tf()`

In [ ]:
# with sess.as_default():
np.set_printoptions(linewidth=150, precision=4)
# print('scatter shape is ', pred_scatt.get_shape())
print('fcn_heatmap_score shape is ', fcn_heatmap_scores.shape)
img = 0
for k in range(4):
    print('Image ', img , '/ Class ',k,' ------------')
    print(fcn_heatmap_scores[img,k,:,4:])


####  Display for visual check - `gt_heatmap_scores`  and `fcn_heatmap_scores` 

In [ ]:
# with sess.as_default():
np.set_printoptions(linewidth=150, precision=6)
# print('scatter shape is ', pred_scatt.get_shape())

img = 0
for i in [1,2,3]:
    for j in range(32):
        print('Image ', img , '/ Class ',i,' ------------')

        print(gt_heatmap_scores[img,i,j])
        print(pred_heatmap_scores[img,i,j])
        print(fcn_heatmap_scores[img,i,j,[0,1,2,3,4,5,11,12,13,14,15]])

### FCN Losses

In [ ]:
def smooth_l1_loss(y_true, y_pred):
    """Implements Smooth-L1 loss.
    y_true and y_pred are typicallly: [N, 4], but could be any shape.
    """
    diff = KB.abs(y_true - y_pred)
    less_than_one = KB.cast(KB.less(diff, 1.0), "float32")
    loss = (less_than_one * 0.5 * diff**2) + (1 - less_than_one) * (diff - 0.5)
    return loss


In [ ]:
##--------------------------------------------------------------------------
## setup input values
input_target = tf.identity(gt_heatmap_scores)
input_pred   = tf.identity(fcn_scores)
# in_tensor = tf.placeholder(tf.float32, shape=[3,4,32,6], name = 'in_tensor')
sess = KB.get_session()
config = model.config
names = ['Dev']
##--------------------------------------------------------------------------


##-----------------------------------------------------------------------
##  FCN loss
##-----------------------------------------------------------------------    
# def fcn_norm_loss_graph(input_target,  input_pred):
with sess.as_default():
    '''
    Generate Loss based on Normalized score in PRED_HEATMAP_SCORES and FCN_HEATMAP_SCORES 
    
    Inputs:            
    gt_heatmap_scores   [batch, num_classes, num_rois, 11 ] --> column 9 contains normalized score.
    pred_heatmap:       [batch, num_classes, num_rois, 16 ] --> column 14 contains normalized score
    '''
    pred_scores   = input_pred[...,14]
    target_scores = input_target[...,9]
    # Reshape for simplicity. Merge first two dimensions into one.
    print('\n>>> fcn_norm_loss_graph ' )
    print('    target_scores shape :', target_scores.shape)
    print('    pred_scores   shape :', pred_scores.shape)    

    target_scores1 = KB.reshape(target_scores, (-1,1))
    print('    target_scores1 shape :', target_scores1.get_shape(), KB.int_shape(target_scores1))        
    pred_scores1   = KB.reshape(pred_scores  , (-1,1))
    print('    pred_scores1  shape :', pred_scores1.get_shape())        

#     # Compute binary cross entropy. If no positive ROIs, then return 0.
#     # shape: [batch, roi, num_classes]
#     # Smooth-L1 Loss
    loss        = KB.switch(tf.size(target_scores1) > 0,
                    smooth_l1_loss(y_true=target_scores1, y_pred=pred_scores1),
                    tf.constant(0.0))
    loss        = KB.mean(loss)
    loss        = tf.reshape(loss, [1, 1], name = 'fcn_normalized_loss')
    print('    loss type is :', type(loss))
    return loss
     

###  Softmax Sparse Cross Entropy Ignoring Last Label -- Used in Keras FCN

In [ ]:
import keras.backend as K 

y_pred = tf.placeholder(dtype=tf.float32, shape=(16,320,320,20))
y_true = tf.placeholder(dtype=tf.float32, shape=(16,320,320,1))
print(K.int_shape(y_pred), K.int_shape(y_true))
y_pred = K.reshape(y_pred, (-1, K.int_shape(y_pred)[-1]))
print(K.int_shape(y_pred))
log_softmax = tf.nn.log_softmax(y_pred)
print(K.int_shape(log_softmax)) 

y_true = K.flatten(y_true)
print(K.int_shape(y_true))

y_true = K.one_hot(tf.to_int32(y_true), K.int_shape(y_pred)[-1]+1)
print(K.int_shape(y_true))

unpacked = tf.unstack(y_true, axis=-1)
print(len(unpacked), unpacked[0].shape)

y_true = tf.stack(unpacked[:-1], axis=-1)
print(K.int_shape(y_true))


cross_entropy = -K.sum(y_true * log_softmax, axis=1)
print(K.int_shape(cross_entropy))

cross_entropy_mean = K.mean(cross_entropy)
print(K.int_shape(cross_entropy_mean))




# import keras.backend as K
# print(K.int_shape(bef_pos)[-1])
# unpacked  = K.flatten(test)
# unpacked.shape

###  Calculate FCN bbox loss

###  Calculate FCN Norm loss

###  Calculate FCN bbox loss

###  Calculate FCN bbox loss

In [ ]:
sess = KB.get_session()
target_bbox_deltas = gt_deltas 
from mrcnn.loss import smooth_l1_loss

In [ ]:
def fcn_bbox_loss_graph(target_bbox_deltas, target_class_ids, fcn_bbox_deltas):
    print('\n>>> fcn_bbox_loss_graph ' )
    print('    target_class_ids  :', target_class_ids.shape)
    print('    fcn_bbox_deltas   :', fcn_bbox_deltas.shape)
    print('    target_bbox_deltas    :', target_bbox_deltas.shape)    

    ## Reshape to merge batch and roi dimensions for simplicity.
    class_array   = KB.reshape(target_bbox_deltas[...,-2]  , (-1, 1))
    tgt_bbox      = KB.reshape(target_bbox_deltas[...,:-2] , (-1, 4))
    pred_bbox     = KB.reshape(fcn_bbox_deltas, (-1, 4))
    print('    reshaped class_array            :', class_array.shape)
    print('    reshaped pred_bbox size         :', pred_bbox.shape)
    print('    reshaped target_bbox size       :', tgt_bbox.shape)    

    ## Only positive ROIs contribute to the loss. And only the right 
    ## class_id of each ROI. Get their indicies.

    pos_ix = tf.where(target_bbox_deltas[...,-2] > 0)

    ## Gather the deltas (predicted and true) that contribute to loss

    # IMPORTANT: THE :-2 IS TO PREVENT ADDITIONAL ELEMENTS FROM BEING COPIED
    y_true = tf.gather_nd(target_bbox_deltas[...,:-2], pos_ix)
    y_pred = tf.gather_nd(fcn_bbox_deltas, pos_ix)
    # print(y_pred.eval(session=sess))
    # print(tf.shape(y_pred).eval(session=sess), tf.shape(y_true).eval(session=sess))    
    print('    y_true shape:', y_true.get_shape())
    print('    y_pred shape:', y_pred.get_shape())


    ## Smooth-L1 Loss
    loss        = KB.switch(tf.size(y_true) > 0,
                    smooth_l1_loss(y_true=y_true, y_pred=y_pred),
                    tf.constant(0.0))
    loss        = KB.mean(loss)
    loss        = KB.reshape(loss, [1, 1])
    return loss

In [ ]:
loss  = fcn_bbox_loss_graph(target_bbox_deltas, target_class_ids, fcn_bbox_deltas)
print(loss.eval(session=sess))

In [ ]:
input_tgt_bbox_deltas = gt_deltas
input_pred_bbox_deltas = fcn_bbox_deltas
input_target_class_ids = target_class_ids
print(input_target_class_ids.shape)
print(input_target_class_ids)
print(input_tgt_bbox_deltas.shape)
# print(input_tgt_bbox_deltas[...,-2])
print(input_tgt_bbox_deltas[0,1,0:10])
class_array = input_tgt_bbox_deltas[...,-2]
print('class array: ', class_array.shape )
print(class_array[0] )
print(input_tgt_bbox_deltas[0])

In [ ]:
sess = KB.get_session()
print('\n>>> fcn_bbox_loss_graph ' )
print('    target_class_ids  :', input_target_class_ids.shape)
print('    fcn_bbox_deltas   :', input_pred_bbox_deltas.shape)
print('    gt_bbox_deltas    :', input_tgt_bbox_deltas.shape)    
# class_array = input_tgt_bbox_deltas[...,-2] 
print('    class-array       :', class_array.shape )
 
cls_arr       = KB.reshape(input_tgt_bbox_deltas[...,-2]  , (-1, 1))
tgt_bbox      = KB.reshape(input_tgt_bbox_deltas[...,:-2] , (-1, 4))
pred_bbox     = KB.reshape(input_pred_bbox_deltas, (-1, 4))
print('    reshaped class_array            :', cls_arr.shape)
print('    reshaped pred_bbox size         :', pred_bbox.shape)
print('    reshaped target_bbox size       :', tgt_bbox.shape)    
print(cls_arr.eval(session = sess))

In [ ]:
# pos_ix = tf.where(input_target_class_ids > 0)
# print(pos_ix.eval(session=sess))
# pos_roi_cls_ids = tf.gather_nd(input_target_class_ids, pos_ix)
# print(pos_roi_cls_ids.eval(session=sess))
# pos_roi_cls_ids = tf.expand_dims(pos_roi_cls_ids, axis = -1)
# print(tf.shape(pos_roi_cls_ids).eval(session=sess))
# ind = tf.concat([tf.to_int32(pos_ix), pos_roi_cls_ids], axis=-1)
# print(ind.eval(session=sess))
# tst_ix = tf.where(cls_arr > 0)
# print(tst_ix.eval(session=sess))

pos_ix = tf.where(input_tgt_bbox_deltas[...,-2] > 0)
print(pos_ix.eval(session=sess))
tgt_pos_cls_deltas = tf.gather_nd(input_tgt_bbox_deltas[...,:-2], pos_ix)
print(tgt_pos_cls_deltas.eval(session=sess))
pred_pos_cls_deltas = tf.gather_nd(input_pred_bbox_deltas, pos_ix)
print(pred_pos_cls_deltas.eval(session=sess))
print(tf.shape(pred_pos_cls_deltas).eval(session=sess), tf.shape(tgt_pos_cls_deltas).eval(session=sess))
# pos_roi_cls_ids = tf.expand_dims(pos_roi_cls_ids, axis = -1)
# print(tf.shape(pos_roi_cls_ids).eval(session=sess))
# ind = tf.concat([tf.to_int32(pos_ix), pos_roi_cls_ids], axis=-1)
# print(ind.eval(session=sess))


In [ ]:
print(input_pred_bbox_deltas.shape)

In [ ]:
# # Reshape to merge batch and roi dimensions for simplicity.
# tgt_class_ids = KB.reshape(target_class_ids, (-1,))
pred_shape = KB.int_shape(pred_bbox_deltas)
tgt_shape = KB.int_shape(tgt_bbox_deltas)
print('    transposed fcn_bbox_deltas   :', pred_shape)
print('    transposed gt_bbox_deltas    :', tgt_shape)    

tgt_bbox      = KB.reshape(tgt_bbox_deltas, (tgt_shape[0],  -1))
pred_bbox     = KB.reshape(pred_bbox_deltas   , (pred_shape[0] ,  -1))
print('    pred_bbox size         :', pred_bbox.shape)
print('    target_bbox size       :', tgt_bbox.shape)    

# # Only positive ROIs contribute to the loss. And only
# # the right class_id of each ROI. Get their indicies.
print(target_class_ids)
positive_roi_ix        = tf.where(target_class_ids > 0)[:,0] 
print(positive_roi_ix.eval(session=sess))
positive_roi_class_ids = tf.cast( tf.gather(target_class_ids, positive_roi_ix), tf.int64)
print(positive_roi_class_ids.eval(session=sess))
# indices                = tf.stack([positive_roi_ix, positive_roi_class_ids], axis=1)

# # Gather the deltas (predicted and true) that contribute to loss
# target_bbox = tf.gather(gt_bbox_deltas, positive_roi_ix)
# pred_bbox   = tf.gather_nd(fcn_bbox_deltas, indices)
# print('    pred_bbox size         :', pred_bbox.shape)
# print('    target_bbox size       :', target_bbox.shape)    

# # Smooth-L1 Loss
# loss        = KB.switch(tf.size(target_bbox) > 0,
                # smooth_l1_loss(y_true=target_bbox, y_pred=pred_bbox),
                # tf.constant(0.0))
loss        = KB.constant(0.0)                    
loss        = KB.mean(loss)
loss        = KB.reshape(loss, [1, 1])
# return loss

### Calculate  mrcnn_bbox_loss

In [ ]:
import keras.backend as K

from mrcnn.utils import apply_box_deltas
from mrcnn.loss  import smooth_l1_loss
target_class_ids   = model_output[5]
target_bbox        = model_output[6]
mrcnn_bbox          = model_output[10] 

# target_bbox      = layers_out[2][0:1]
# mrcnn_class        = model_output[9]
# mrcnn_bbox       = layers_out[10][0:1]
mrcnn_class_ids  = np.argmax(model_output[9],axis = -1)     # mrcnn_class_ids
print('target_class_ids ', target_class_ids.shape)
print(target_class_ids[0])
print('target_bbox: ',target_bbox.shape)
print(target_bbox[0])
print('mrcnn_bbox : ',mrcnn_bbox.shape)
print(mrcnn_bbox[0])


In [ ]:
print('target_class_ids', target_class_ids.shape)
print(target_class_ids)  # tgt_class_ids
# print(' class with max probability', mrcnn_class_ids.shape)
# print(mrcnn_class_ids)
print('target_bboxes shape', target_bbox.shape)
# print(target_bbox)  # tgt_bounding boxes
print('mrcnn_bboxes shape',mrcnn_bbox.shape)
# print(mrcnn_bbox)  #mrcnn_bboxes
pred_bbox = mrcnn_bbox

In [ ]:
# calc mrcnn_bbox_loss
target_class_ids = K.reshape(target_class_ids, (-1,))
print('target_class_ids shape: ', target_class_ids.shape)
print(target_class_ids.eval(session=sess))

target_bbox      = K.reshape(target_bbox, (-1, 4))
print('target_bbox: ', target_bbox.shape)

pred_bbox        = K.reshape(pred_bbox, (-1, pred_bbox.shape[2], 4))
print('pred_bbox : ', pred_bbox.shape)

In [ ]:
# test_ix        = tf.where(target_class_ids > 0)
# print('test_ix',test_ix.shape)
# print(test_ix.eval(session=sess))

positive_roi_ix        = tf.where(target_class_ids > 0)[:, 0]
print('postivie roi ix')
print(positive_roi_ix.eval(session=sess))

positive_roi_class_ids = tf.cast( tf.gather(target_class_ids, positive_roi_ix), tf.int64)
print('postivie roi class_ids')
print(positive_roi_class_ids.eval(session=sess))

indices                = tf.stack([positive_roi_ix, positive_roi_class_ids], axis=1)
print('indices for gathering from mrcnn_bbox')
print(indices.eval(session=sess))


target_bbox2 = tf.gather(target_bbox, positive_roi_ix)
print('target bbox 2', tf.shape(target_bbox2).eval(session=sess))
print(target_bbox2.eval(session=sess))

pred_bbox2   = tf.gather_nd(pred_bbox, indices)
print('pred_bbox 2 ', tf.shape(pred_bbox2).eval(session=sess))
print(pred_bbox2.eval(session=sess))

In [ ]:
print('tf.size ',tf.size(target_bbox2).eval(session=sess))

diff = K.abs(target_bbox2 - pred_bbox2)
print(diff.eval(session=sess))

less_than_one = K.cast(K.less(diff, 1.0), "float32")
# print(less_than_one.eval())

loss = (less_than_one * 0.5 * diff**2) + (1 - less_than_one) * (diff - 0.5)
# print( (1-less_than_one).eval())



# loss        = K.switch(tf.size(target_bbox) > 0,
#                 smooth_l1_loss(y_true=target_bbox, y_pred=pred_bbox),
#                 tf.constant(0.0))
print(loss.eval())
sumloss = K.sum(loss)
print(sumloss.eval())
print((sumloss/40).eval())
meanloss        = K.mean(loss)
print(meanloss.eval())

###  Calculate mrcnn_class_loss

In [ ]:
import keras.backend as K

from mrcnn.utils import apply_box_deltas
from mrcnn.loss  import smooth_l1_loss

target_class_ids = layers_out[1][0:1]
pred_class_logits = layers_out[8][0:1]
active_class_ids    = np.array([1,1,1,1])

# mrcnn_class_ids  = np.argmax(layers_out[9][0:1],axis = -1)     # mrcnn_class_ids

print(' target_class_ids', target_class_ids.shape)
print(target_class_ids)  # tgt_class_ids
print(' class logits', pred_class_logits.shape)
print(pred_class_logits)
print(' active, class_ids ', active_class_ids.shape)
print(active_class_ids)  # tgt_bounding boxes

pred_class_ids = tf.argmax(pred_class_logits, axis=2)
print(pred_class_ids.eval())  #mrcnn_bboxes
mrcnn_class_ids  = np.argmax(layers_out[9][0:1],axis = -1)     # mrcnn_class_ids
print(mrcnn_class_ids)
# pred_bbox = mrcnn_bbox
pred_active = tf.to_float(tf.gather(active_class_ids, pred_class_ids))
print(pred_active.eval())
# calc mrcnn_bbox_loss
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
       labels=target_class_ids, logits=pred_class_logits)
print(loss.eval())

loss = loss * tf.to_float(pred_active)
print(loss.eval())

print(tf.reduce_sum(loss).eval())
print(tf.reduce_sum(pred_active).eval())
loss = tf.reduce_sum(loss) / tf.reduce_sum(pred_active)
print(loss.eval())

###  Calculate mrcnn_mask_loss

In [ ]:
import keras.backend as K

from mrcnn.utils import apply_box_deltas
from mrcnn.loss  import smooth_l1_loss

target_class_ids    = layers_out[1][0:3]
target_masks        = layers_out[3][0:3]
pred_masks          = layers_out[11][0:3]
# mrcnn_class_ids  = np.argmax(layers_out[9][0:1],axis = -1)     # mrcnn_class_ids
print('    target_class_ids shape :', target_class_ids.shape)
print('    target_masks     shape :', target_masks.shape)
print('    pred_masks       shape :', pred_masks.shape)    


target_class_ids = K.reshape(target_class_ids, (-1,))
print('    target_class_ids shape :', target_class_ids.shape, '\n', target_class_ids.eval())

mask_shape       = tf.shape(target_masks)
print('    mask_shape       shape :', mask_shape.shape, mask_shape.eval())    

target_masks     = K.reshape(target_masks, (-1, mask_shape[2], mask_shape[3]))
print('    target_masks     shape :', tf.shape(target_masks).eval())        

pred_shape       = tf.shape(pred_masks)
print('    pred_shape       shape :', pred_shape.shape, pred_shape.eval())        

pred_masks       = K.reshape(pred_masks, (-1, pred_shape[2], pred_shape[3], pred_shape[4]))
print('    pred_masks       shape :', tf.shape(pred_masks).eval())        


pred_masks = tf.transpose(pred_masks, [0, 3, 1, 2])
print('    pred_masks       shape :', tf.shape(pred_masks).eval())        

# Only positive ROIs contribute to the loss. And only
# the class specific mask of each ROI.
positive_ix        = tf.where(target_class_ids > 0)[:, 0]
positive_class_ids = tf.cast(tf.gather(target_class_ids, positive_ix), tf.int64)
indices            = tf.stack([positive_ix, positive_class_ids], axis=1)
print(indices.eval())



y_true = tf.gather(target_masks, positive_ix)
print('     y_true shape:', tf.shape(y_true).eval())
y_pred = tf.gather_nd(pred_masks, indices)
print('     y_pred shape:', tf.shape(y_pred).eval())

loss = K.switch(tf.size(y_true) > 0,
                K.binary_crossentropy(target=y_true, output=y_pred),
                tf.constant(0.0))
print(tf.shape(loss).eval())

loss = K.mean(loss)
print('     final loss shape:', tf.shape(loss).eval())
print(loss.eval())
loss = K.reshape(loss, [1, 1])
print('     final loss shape:', tf.shape(loss).eval())
print(loss.eval())

### Calculate a pixel loss on fcn_gaussian and gt_gaussian 

In [ ]:
import keras.backend as K

from mrcnn.utils import apply_box_deltas
from mrcnn.loss  import smooth_l1_loss
pred_masks          = layers_out[12][0:3]   # fcn_predictions 
target_masks        = layers_out[19][0:3]   # gt_gaussians

print('    target_masks     shape :', tf.shape(target_masks).eval())
print('    pred_masks       shape :', tf.shape(pred_masks).eval())    

diff = K.abs(target_masks - pred_masks)
print(tf.shape(diff).eval())

less_than_one = K.cast(K.less(diff, 1.0), "float32")
print('   less_than_one     shape :', tf.shape(less_than_one).eval(), K.sum(less_than_one).eval())

more_than_one = 1 - less_than_one
print('   more_than_one     shape :', tf.shape(more_than_one).eval(), K.sum(more_than_one).eval())
# loss = (less_than_one * 0.5 * diff**2) + (1 - less_than_one) * (diff - 0.5)

loss = (less_than_one * 0.5 * diff**2) + (more_than_one * (diff - 0.5))
print(tf.shape(loss).eval())

# loss = K.switch(tf.size(y_true) > 0,
#                 K.binary_crossentropy(target=y_true, output=y_pred),
#                 tf.constant(0.0))
meanloss = K.mean(loss)
print(tf.shape(meanloss).eval())
print(meanloss.eval())
# loss = K.reshape(loss, [1, 1])
# print('     final loss shape:', loss.get_shape())
# return loss


mask_shape       = tf.shape(target_masks)
print('    mask_shape       shape :', tf.shape(mask_shape).eval())    

target_masks     = K.reshape(target_masks, (-1, mask_shape[1], mask_shape[2]))
print('    target_masks     shape :', tf.shape(target_masks).eval())        

pred_shape       = tf.shape(pred_masks)
print('    pred_shape       shape :', tf.shape(pred_shape).eval())        

pred_masks       = K.reshape(pred_masks, (-1, pred_shape[1], pred_shape[2]))
print('    pred_masks       shape :', tf.shape(pred_masks).eval())

# Permute predicted masks to [N, num_classes, height, width]
# diff = K.abs(target_masks - pred_masks)
# print(tf.shape(diff).eval())

# less_than_one = K.cast(K.less(diff, 1.0), "float32")
# print(tf.shape(less_than_one).eval())

# loss = (less_than_one * 0.5 * diff**2) + (1 - less_than_one) * (diff - 0.5)
# print(tf.shape(loss).eval())

# meanloss = K.mean(loss)
# print(tf.shape(meanloss).eval())
# print(meanloss.eval())

loss = K.switch(tf.size(target_masks) > 0,
                smooth_l1_loss(y_true=target_masks, y_pred=pred_masks),
                tf.constant(0.0))
loss = K.mean(loss)
loss = K.reshape(loss, [1, 1])
print('     final loss shape:', loss.get_shape())
print(loss.eval())

In [ ]:
img  = 0
class_probs = layers_out[9][img]   # mrcnn_class
deltas      = layers_out[10][img]       # mrcnn_bbox

print(class_probs.shape)
print('class probabilities')
print(class_probs)
class_ids = np.argmax(layers_out[9][img],axis = 1)     # mrcnn_class_ids
print(' class with max probability')
print(class_ids)


# layers_out[10][2,0,3]
print('deltas.shape :', deltas.shape)
print(deltas[0:4])

deltas_specific = deltas[np.arange(32),class_ids]
print('deltas of max prob class: ', deltas_specific.shape)
print(deltas_specific[0:5])
output_rois = layers_out[0][img]*[128,128,128,128]
print('output_rois: ', output_rois.shape)
print(output_rois[0:])

refined_rois    = apply_box_deltas(output_rois, deltas_specific * config.BBOX_STD_DEV)
print('refined rois: ',refined_rois.shape)
print(refined_rois)

In [ ]:
img = 0
cls = 0
fcn_out = layers_out[12][img]
fcn_sum = np.sum(fcn_out, axis=(0,1))
print(fcn_sum)
for cls in range(4):
    print('min :', np.min(fcn_out[:,:,cls]), 'max :', np.max(fcn_out[:,:,cls]), )

In [ ]:
print(train_batch_x[4][2])
print(train_batch_x[5][2]/[128,128,128,128])